In [24]:
import numpy as np
from scipy import sparse
from sklearn.metrics.cluster import contingency_matrix

In [18]:
# ground true 真实分类情况
label_true = np.array([1, 2, 1, 1, 3])
# 预测分类结果
# label_pred = [1, 3, 1, 2, 2]
label_pred = np.array([1, 2, 1, 1, 3])

In [14]:
# 可能性矩阵
contingency = contingency_matrix(label_true, label_pred)
print('可能性矩阵(contingency matrix)：')
print(contingency)

可能性矩阵(contingency matrix)：
[[3 0 0]
 [0 1 0]
 [0 0 1]]


上面 contingency matrix 得到的结果含义如下：

$
\begin{bmatrix}
    C_{11} & C_{12} & C_{13} \\
    C_{21} & C_{22} & C_{23} \\
    C_{31} & C_{32} & C_{13} \\
\end{bmatrix}
$

其中 $C_{ij}$ 中的 $i$ 表示 `label_true` 的分类 $\{1,2,3\}$, $j$ 表示 `label_pred` 的分类 $\{1,2,3\}$

In [15]:
# 非零行索引，非零列索引，非零数据值
# contingency[nzx, nzy] 能得到所有非零的数据
nzx, nzy, nz_val = sparse.find(contingency)

print(f'非零行索引: {nzx}')
print(f'非零列索引: {nzy}')
print(f'非零数据值: {nz_val}')

非零行索引: [0 1 2]
非零列索引: [0 1 2]
非零数据值: [3 1 1]


In [49]:
# 求可能性矩阵的合计值
contingency_sum = contingency.sum()
# 按行合计可能性矩阵
pi = np.ravel(contingency.sum(axis=1))
# 按列合计可能性矩阵
pj = np.ravel(contingency.sum(axis=0))
# 非零值的对数
log_contingency_nm = np.log(nz_val)
# 分类混淆结果的概率
contingency_nm = nz_val / contingency_sum
# pi = [3 1 1] nzx = [0 0 1 2] => pi.take(nzx) = [3 3 1 1]
# pj = [2 2 1] nzy = [0 1 2 1] => pj.take(nzy) = [2 2 1 2]
outer = (pi.take(nzx).astype(np.int64, copy=False)
         * pj.take(nzy).astype(np.int64, copy=False))
log_outer = -np.log(outer) + np.log(pi.sum()) + np.log(pj.sum())
mi = (contingency_nm * (log_contingency_nm - np.log(contingency_sum))
      + contingency_nm * log_outer)

print(f'求可能性矩阵的合计值: {contingency_sum}')
print(f'按行合计可能性矩阵: {pi}')
print(f'按列合计可能性矩阵: {pj}')
print(f'非零值的对数(2-base): {log_contingency_nm}')
print(f'分类混淆结果的概率: {contingency_nm}')
print(f'outer: {outer}')
print(f'log_outer: {log_outer}')
print(f'互信息 mi: {mi.sum()}')

求可能性矩阵的合计值: 5
按行合计可能性矩阵: [3 1 1]
按列合计可能性矩阵: [3 1 1]
非零值的对数(2-base): [1.09861229 0.         0.        ]
分类混淆结果的概率: [0.6 0.2 0.2]
outer: [9 1 1]
log_outer: [1.02165125 3.21887582 3.21887582]
互信息 mi: 0.9502705392332345


$contingency\_nm = \frac{可能性矩阵的非空值}{可能性矩阵的合计值}$

这个值的意思是 **真实分类** 和 **预测分类** 的混淆结果 $C_{ij}$ 与整个混淆矩阵的合计值的比, 可以看做是概率

互信息 mutual information 公式

$
I(X;Y) = H(Y) - H(Y|X)
$

In [48]:
entropy_true = label_true / label_true.sum()
entropy_true = -(entropy_true * np.log(entropy_true)).sum()

entropy_pred = label_pred / label_pred.sum()
entropy_pred = -(entropy_pred * np.log(entropy_pred)).sum()

entropy_true, entropy_pred

(1.4941751382893083, 1.4941751382893083)

In [27]:
label_true

array([1, 2, 1, 1, 3])

In [31]:
a = np.array([3/5, 1/5, 1/5])
a

array([0.6, 0.2, 0.2])

In [32]:
k = -(a * np.log2(a)).sum()
k

1.3709505944546687

In [42]:
w = np.unique(label_pred, return_inverse=True)
w

(array([1, 2, 3]), array([0, 1, 0, 0, 2]))

In [43]:
pi = np.bincount(w[1])
pi

array([3, 1, 1])

In [47]:
pi_sum = np.sum(pi)
-np.sum((pi / pi_sum) * (np.log(pi) - np.log(pi_sum)))

0.9502705392332345

In [45]:
from sklearn.metrics.cluster import mutual_info_score

In [46]:
mutual_info_score(label_true, label_pred)

0.9502705392332345